<a href="https://colab.research.google.com/github/LTayrone/KMeans-Analise/blob/main/Programa%C3%A7%C3%A3o_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Resolução de problema de Programação Linear**

**Autor:  Lucas Tayrone**

https://www.linkedin.com/in/lucastayrone/

https://github.com/ltayrone

# O que é programação linear?
A programação linear trata da maximização (ou minimização) de uma função objetivo linear, sujeita a restrições lineares, onde todas as variáveis ​​de decisão são contínuas. Ou seja, nenhuma variável discreta é permitida. O objetivo linear e as restrições devem consistir em expressões lineares.

## **Exemplo: um problema de produção** 

Neste tópico, você analisará um problema de produção simples em termos de variáveis ​​de decisão, função objetivo e restrições.


### **Descrição do problema:** 

### **Processo de Fabricação de Aços**

Uma Fabrica metalurgica e siderúrgica produz e vende dois tipos de cilindros de aço, ou seja, Lâminado e Trefilado.

Cada tipo de cilindro é produzido e revestido pela fabrica. O objetivo é maximizar o lucro, e a fabrica precisa produzir pelo menos 5000 unidades de cada tipo de cilindro.

Existe limites em termos de capacidade de produção da fabrica, e eles precisam calcular o número ideal de cada tipo de cilindro para produzir, sem exceder a capacidade da fábrica.

Para chegar a um modelo descritivo, consideremos as seguintes questões:




*   Quais são as variáveis ​​de decisão?
*   Qual é o objetivo?
*   Quais são as restrições?

Produção de cilindros: um modelo descritivo 
Um possível modelo descritivo do problema de produção de cilindros é o seguinte:

### **Informações:**

Custo:

**LamProduction** = R$2.390

**TrefProduction** = R$3.100

Tempo de Produção: 

**LamProduction** = 0.2h
**TrefProduction** = 0.4h

Tempo de REvestimento: 

**LamProduction** = 0.5h
**TrefProduction** = 0.4h


### **Variáveis ​​de decisão:**
Número de cilindros lâminados a produzir (LamProduction)

Número de cilindros Trefilados produzidos (TrefProduction)

### **Objetivo:** Maximizar o lucro

### **Restrições:**

O **LamProduction** deve ser maior ou igual a 5000.

O **TrefProduction** deve ser maior ou igual a 5000.

O tempo de fabricação para **LamProduction** mais o tempo de fabricação para **TrefProduction** não deve exceder 9000 horas.

O tempo de revestimento para **LamProduction** mais o tempo de revestimento para **TrefProduction** não deve exceder 5600 horas.

In [1]:
#instalando o IBM® Decision Optimization e o CPLEX
!pip install cplex
!pip install docplex

In [2]:
# importando o modelo do docplex.mp
from docplex.mp.model import Model

# creando uma instancia com o nome:
m = Model(name='cilindro_producao')

In [3]:
# por padrão todas as variaveis do Docplex tem limite inferior 0 e superior infinito
lam = m.continuous_var(name='lam')
tref = m.continuous_var(name='tref')

## **Restrições:**

In [4]:
# incluindo as restrições
# restrição #1: laminado não pode ser menor que 5000
m.add_constraint(lam >= 6000)

# restrição #2: trefilado não pode ser menor que 5000
m.add_constraint(tref >= 5000)

# restrição #3: produção tempo limite
ct_prod = m.add_constraint( 0.2 * lam + 0.4 * tref <= 9000) #225 funcionarios trabalhando 40 horas

# restrição #4: revestimento tempo limite
ct_rev = m.add_constraint( 0.5 * lam + 0.4 * tref <= 5600) #140 funcionarios trabalhando 40 horas

### **Objetivo:**

In [5]:
m.maximize(2390 * lam + 3100 * tref)
m.print_information()

Model: cilindro_producao
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 4
   - linear=4
 - parameters: defaults
 - objective: maximize
 - problem type is: LP


### **Resolução do Modelo**

In [6]:
s = m.solve()
m.print_solution()



objective: 34490000.000
  lam=6000.000
  tref=6500.000


# **Relaxando restrições**

No caso dos modelos de Progrmação Linear, o termo “relaxamento” refere-se à alteração do lado direito da restrição para permitir alguma violação da restrição original.

Por exemplo:

No ultimo problema apresentado, ao substituimos as variaveis de decisão na restrição, vimos que a mesma atinge sua capacidade maxima. Sendo ela um gargalo para nosso problema.

( 0.5 * **(5000)lam** + 0.4 * **(7750)tref** <= 5600) #140 funcionarios trabalhando 40 horas

# **Exemplo de condição de Relaxamento:**

Após uma discurção, o time de planejamento estudou a possibilidade de utilizar o sistema de hora extra para aumentar a capacidade do processo de revestimento:

Limite pelo Ministerio do Trabalho 10h semanais, para 140 funcionario, totalizam-se 1400h, o custo da hora extra é de R$15H. 

### **Hora extra pelo DOcplex**

In [7]:
extra = m.continuous_var(name='extra', ub=1400)

**Modificando a restrição de revestimento**

In [8]:
ct_rev.rhs =  5600 + extra

**Incluindo a penalidade de custo da hora extra**

In [9]:
m.maximize(2390 * lam + 3100 * tref - 15 * extra)
m.print_information()

Model: cilindro_producao
 - number of variables: 3
   - binary=0, integer=0, continuous=3
 - number of constraints: 4
   - linear=4
 - parameters: defaults
 - objective: maximize
 - problem type is: LP


**Resolvendo novamente o problema**

In [10]:
s2 = m.solve()
m.print_solution()

objective: 45319000.000
  lam=6000.000
  tref=10000.000
  extra=1400.000
